In [1]:
import argparse
import gym
import numpy as np
from itertools import count
from syft.controller import tensors, models

import syft
import syft.interfaces.torch.actual_torch as actual_torch
import syft.interfaces.torch as torch
import syft.interfaces.torch.nn as nn
import syft.interfaces.torch.nn.functional as F
import syft.interfaces.torch.optim as optim
from syft.interfaces.torch.autograd import Variable
from syft.interfaces.torch.distributions import Categorical

import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical

gamma = 0.9
seed = 543
render = False
log_interval = 100

env = gym.make('CartPole-v0')
env.seed(seed)
torch.manual_seed(seed)

class Policy(nn.Module):
    
    def __init__(self):
        super(Policy, self).__init__()
        self.affine = nn.Linear(4, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        action_scores = self.affine(x)
        return F.softmax(action_scores, dim=1)
    
policy = Policy()

def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(Variable(state))
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.data[0]

[2017-12-29 20:37:35,375] Making new env: CartPole-v0


In [2]:
state = env.reset()

In [3]:
action = select_action(state)

In [4]:
out = syft.FloatTensor(torch.rand(10,2,2).numpy())

In [10]:
ind = syft.IntTensor([1,2,3,4,5])

In [14]:
subset = out.index_select(ind,0)

Exception: Unity Error: System.Collections.Generic.KeyNotFoundException: The given key was not present in the dictionary.
  at System.ThrowHelper.ThrowKeyNotFoundException () [0x00000] in /Users/builduser/buildslave/mono/build/mcs/class/referencesource/mscorlib/system/throwhelper.cs:68 
  at System.Collections.Generic.Dictionary`2[TKey,TValue].get_Item (TKey key) [0x0001e] in /Users/builduser/buildslave/mono/build/mcs/class/referencesource/mscorlib/system/collections/generic/dictionary.cs:183 
  at OpenMined.Syft.Tensor.Factories.FloatTensorFactory.Get (System.Int32 id) [0x00008] in /Users/atrask/Laboratory/openmined/OpenMined/UnityProject/Assets/OpenMined/Syft/Tensor/Factories/FloatTensorFactory.cs:24 
  at OpenMined.Network.Controllers.SyftController.processMessage (System.String json_message) [0x000d5] in /Users/atrask/Laboratory/openmined/OpenMined/UnityProject/Assets/OpenMined/Network/Controllers/SyftController.cs:86 

In [12]:
subset

[[[ 0.2941406   0.9989439 ]
  [ 0.5207259   0.3590897 ]]

 [[ 0.3861234   0.8082908 ]
  [ 0.2876182   0.5556896 ]]

 [[ 0.1953882   0.9048445 ]
  [ 0.226079    0.5687689 ]]

 [[ 0.3615693   0.5218327 ]
  [ 0.1339474   0.6470152 ]]

 [[ 0.575147    0.09562236]
  [ 0.8301317   0.8498407 ]]]
[syft.FloatTensor:10 grad:None size:5x2x2 c:[] p:[9] init:view_5_2_2]

	-----------creators-----------
	[syft.FloatTensor:9 grad:None size:1x5x4 c:[10] p:[1] init:index_select_dim:0_int-id:2]
	------------------------------



In [13]:
out.index_add(ind,0,subset.neg())

[[[ 0.3051162   0.9820263 ]
  [ 0.1147337   0.7638507 ]]

 [[ 0.          0.        ]
  [ 0.          0.        ]]

 [[ 0.          0.        ]
  [ 0.          0.        ]]

 [[ 0.          0.        ]
  [ 0.          0.        ]]

 [[ 0.          0.        ]
  [ 0.          0.        ]]

 [[ 0.          0.        ]
  [ 0.          0.        ]]

 [[ 0.7655473   0.07898402]
  [ 0.857523    0.01522237]]

 [[ 0.2545201   0.9170496 ]
  [ 0.3031685   0.1795458 ]]

 [[ 0.1431879   0.3000262 ]
  [ 0.5735351   0.4295186 ]]

 [[ 0.4836702   0.3399992 ]
  [ 0.1947275   0.1342559 ]]]
[syft.FloatTensor:14 grad:None size:10x2x2 c:[] p:[13] init:view_10_2_2]

	-----------creators-----------
	[syft.FloatTensor:13 grad:None size:1x10x4 c:[14] p:[1] init:index_add_dim:0_2_11]
	------------------------------



In [9]:
out

[[[ 0.3051162   0.9820263 ]
  [ 0.1147337   0.7638507 ]]

 [[ 0.2941406   0.9989439 ]
  [ 0.5207259   0.3590897 ]]

 [[ 0.3861234   0.8082908 ]
  [ 0.2876182   0.5556896 ]]

 [[ 0.1953882   0.9048445 ]
  [ 0.226079    0.5687689 ]]

 [[ 0.3615693   0.5218327 ]
  [ 0.1339474   0.6470152 ]]

 [[ 0.575147    0.09562236]
  [ 0.8301317   0.8498407 ]]

 [[ 0.7655473   0.07898402]
  [ 0.857523    0.01522237]]

 [[ 0.2545201   0.9170496 ]
  [ 0.3031685   0.1795458 ]]

 [[ 0.1431879   0.3000262 ]
  [ 0.5735351   0.4295186 ]]

 [[ 0.4836702   0.3399992 ]
  [ 0.1947275   0.1342559 ]]]
[syft.FloatTensor:1 grad:None size:10x2x2 c:[2, 3, 7] p:[] init:]

	-----------children-----------
	[syft.FloatTensor:2 grad:None size:1x10x4 c:[] p:[1] init:view_1_10_4]
	[syft.FloatTensor:3 grad:None size:1x5x4 c:[4] p:[1] init:index_select_dim:0_int-id:1]
	[syft.FloatTensor:7 grad:None size:1x10x4 c:[8] p:[1] init:index_add_dim:0_1_5]
	------------------------------



In [18]:
optimizer = optim.SGD(policy.parameters(), lr=0.15)

def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + np.finfo(np.float32).eps)
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]



running_reward = 10
for i_episode in count(1):
    state = env.reset()
    for t in range(10000):  # Don't infinite loop while learning
        action = select_action(state)
        state, reward, done, _ = env.step(action)
        if render:
            env.render()
        policy.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode()
    if i_episode % log_interval == 0:
        print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
            i_episode, t, running_reward))
    if running_reward > env.spec.reward_threshold:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!".format(running_reward, t))
        break


[2017-12-25 14:32:57,551] Making new env: CartPole-v0


Episode 100	Last length:    98	Average length: 66.12
Episode 200	Last length:    17	Average length: 85.61
Episode 300	Last length:    49	Average length: 80.06
Episode 400	Last length:   169	Average length: 138.29
Episode 500	Last length:    41	Average length: 135.51
Episode 600	Last length:    88	Average length: 126.97
Episode 700	Last length:   177	Average length: 163.88
Episode 800	Last length:   126	Average length: 179.32
Episode 900	Last length:    63	Average length: 138.71
Episode 1000	Last length:   199	Average length: 139.44
Episode 1100	Last length:   199	Average length: 176.30
Episode 1200	Last length:   199	Average length: 189.80
Solved! Running reward is now 195.00467036348365 and the last episode runs to 199 time steps!
